# 目标

    使用tensorflow中听的inceptionv3，进行图像分类

In [1]:
import tensorflow as tf
import numpy as np
import os

E:\Python\PY3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
# 注意这里的斜杠需要是正向斜杠 /
model_dir='E:/Projects/00_MyLab/12_tensorflow/inception_model'
image='E:/Projects/00_MyLab/12_tensorflow/images/husky.jpg'

In [21]:
'''
整个类的目标：

    1、读取 分类序号-类别id，以及类别id-类别名称这两个文件，并转化为字典。
    2、最后生成 分类序号-类别名称这个字典。
    3、提供 ‘使用分类序号查询类别名称’ 功能
'''
class NodeLookUp(object):
    def __init__(self,label_lookup_path=None,uid_lookup_path=None):
        if not label_lookup_path:
            # 加载label的查询文件，即imagenet_2012_challenge_label_map_proto.pbtxt
            label_lookup_path = os.path.join(model_dir,
                                             'imagenet_2012_challenge_label_map_proto.pbtxt')
        if not uid_lookup_path:
            # 加载label可读化文件，将uid转换为单词句子
            uid_lookup_path=os.path.join(model_dir,
                                         'imagenet_synset_to_human_label_map.txt')
        
        # 这里很迷啊，为什么能够在初始化方法里面直接调用下面的方法？
        # 我记得java里面认为初始化方法被调用的时候，下面的方法还没有被创建好啊…
        self.node_lookup = self.load(label_lookup_path,uid_lookup_path)
    
    def load(self,label_lookup_path,uid_lookup_path):
        if not tf.gfile.Exists(uid_lookup_path):
            # 检查地址存在与否
            tf.logging.fatal('File does not exist %s',uid_lookup_path)
        if not tf.gfile.Exists(label_lookup_path):
            tf.logging.fatal('File does not exist %s',label_lookup_path)
            
        # 读取uid里面所有的line
        proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
        
        # 创建字典用来存 id-word的映射关系
        uid_to_human={}
        for line in proto_as_ascii_lines:
            line=line.strip('\n')
            parse_items = line.split('\t')
            # 获取编码
            uid = parse_items[0]
            # 获取名称
            human_string=parse_items[1]
            # 存入字典
            uid_to_human[uid]=human_string
            
        # 读取分类字符串和uid对应的文件
        proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
        # 同样创建字典保存关系
        node_id_to_uid={}
        for line in proto_as_ascii:
            # 按行取内容
            if line.startswith('  target_class:'):
                target_class = int(line.split(': ')[1])
            if line.startswith('  target_class_string:'):
                target_class_string = line.split(': ')[1]
                node_id_to_uid[target_class]=target_class_string[1:-2]
        
        # 然后把第二个字典的key，与第一个字典的value在对应起来
        node_id_to_name={}
        for key,val in node_id_to_uid.items():
            if val not in uid_to_human: # 这里是指第二个dict的value，不在第一个dict的key中的话
                tf.logging.fatal('Failed to locate: %s',val)
            name = uid_to_human[val]
            node_id_to_name[key] = name
        
        return node_id_to_name

    def id_to_string(self,node_id):
        if node_id  not in self.node_lookup:
            return ''
        return self.node_lookup[node_id]

In [22]:
'''
需要取了解一下tf存储模型的方式：仅参数，or 参数+图，以及相互的恢复方法

这个方法就是恢复模型的方法
'''
def create_graph():
    # 还有这个tf.gfile是个啥
    with tf.gfile.FastGFile(os.path.join(model_dir,
                                         'classify_image_graph_def.pb'),
                            'rb') as f:
        graph_def = tf.GraphDef() # 这是个啥
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def,name='')

In [23]:
image_data = tf.gfile.FastGFile(image,'rb').read()

In [24]:
'''
问题1：
    平常我们跑的都是节点，这里说拿的是张量，这两个有什么区别吗？
'''
create_graph()
with tf.Session() as sess:
    # 拿到最后一层输出的张量
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    # 输入图像数据
    predictions = sess.run(softmax_tensor,{'DecodeJpeg/contents:0':image_data})
    # 将结果转化为1维数据
    predictions = np.squeeze(predictions)
    # 新建一个刚刚写的id类
    node_lookup = NodeLookUp()
    # 取出预测值较大的前五个索引
    top_5 = predictions.argsort()[-5:][::-1]
    for node_id in top_5:
        # 获取分类名称
        human_string = node_lookup.id_to_string(node_id)
        # 获取置信度
        score = predictions[node_id]
        # 打印结果
        print('%s (score = %.5f)' % (human_string, score))

Eskimo dog, husky (score = 0.61329)
Siberian husky (score = 0.22319)
dogsled, dog sled, dog sleigh (score = 0.00641)
malamute, malemute, Alaskan malamute (score = 0.00371)
Norwegian elkhound, elkhound (score = 0.00184)
